In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Conv2DTranspose ,Flatten, LeakyReLU, Reshape
from keras.models import Sequential , Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/cancer detection/hmnist_28_28_L.csv')

In [ ]:
ds.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel0775,pixel0776,pixel0777,pixel0778,pixel0779,pixel0780,pixel0781,pixel0782,pixel0783,label
0,169,171,170,177,181,182,181,185,194,192,...,184,186,185,180,157,140,140,159,165,2
1,19,57,105,140,149,148,144,155,170,170,...,172,175,160,144,114,89,47,18,18,2
2,155,163,161,167,167,172,155,152,165,175,...,163,178,157,166,167,148,141,136,115,2
3,25,71,116,139,136,153,148,161,172,162,...,125,135,138,137,111,71,32,16,16,2
4,129,162,181,196,205,208,205,213,225,224,...,210,197,172,190,195,193,181,147,88,2


In [ ]:
ds['label'].value_counts()

4    6705
6    1113
2    1099
1     514
0     327
5     142
3     115
Name: label, dtype: int64

In [ ]:
X = ds.drop('label', axis=1)
y= ds['label']

In [ ]:
resample = SMOTETomek(tomek=TomekLinks(sampling_strategy='not majority'))
X, y = resample.fit_resample(X, y)

In [ ]:
scl=StandardScaler()
X = scl.fit_transform(X)

In [ ]:
X_r = np.array(X)
X_r = X.reshape(-1, 28, 28, 1)
print(X.shape)
print(y.value_counts())

(46929, 784)
2    6705
3    6705
5    6705
1    6705
0    6705
4    6702
6    6702
Name: label, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Generator model
latent_dim = 100
num_classes = 7

generator_input = keras.Input(shape=(latent_dim,))

generator_dense = Dense(7 * 7 * 128)(generator_input)

generator_activation = LeakyReLU()(generator_dense)

generator_reshape = Reshape((7, 7, 128))(generator_activation)

generator_conv1 = Conv2DTranspose(128, 4, strides=2, padding="same")(generator_reshape)

generator_activation2 = LeakyReLU()(generator_conv1)

generator_conv2 = Conv2DTranspose(128, 4, strides=2, padding="same")(generator_activation2)

generator_activation3 = LeakyReLU()(generator_conv2)

generator_output = Conv2D(1, 7, activation="softmax", padding="same")(generator_activation3)

generator = Model(generator_input, generator_output)

generator.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_5 (Dense)             (None, 6272)              633472    
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 6272)              0         
                                                                 
 reshape_3 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 14, 14, 128)      262272    
 ranspose)                                                       
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                           

In [ ]:
# Discriminator model
discriminator_input = Input(shape=(28, 28, 1))

discriminator_conv1 = Conv2D(64, 3)(discriminator_input)

discriminator_activation = LeakyReLU()(discriminator_conv1)

discriminator_conv2 = Conv2D(128, 3)(discriminator_activation)

discriminator_flatten = Flatten()(discriminator_conv2)

discriminator_dropout = Dropout(0.4)(discriminator_flatten)

discriminator_output = Dense(1, activation="sigmoid")(discriminator_dropout)

discriminator = Model(discriminator_input, discriminator_output)

discriminator.summary()


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 26, 26, 64)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 flatten_2 (Flatten)         (None, 73728)             0         
                                                                 
 dropout_2 (Dropout)         (None, 73728)             0         
                                                                 
 dense_6 (Dense)             (None, 1)                 7372

In [ ]:
# GAN model
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)

In [ ]:
# Compile models
discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002), loss="binary_crossentropy")
gan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002), loss="binary_crossentropy")

In [ ]:
# Training loop
batch_size = 128
epochs = 100

In [ ]:
df1 =  pd.DataFrame(X_train.reshape(X_train.shape[0], -1))
df2 = pd.DataFrame(y_train)
print(df1.shape)
print(df2.shape)

(37543, 784)
(37543, 1)


In [ ]:
X_train = np.array(df1)
y_train =np.array(df2)

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
print(X_train.shape)

(37543, 28, 28, 1)


In [ ]:
for epoch in range(epochs):
    # Train discriminator
    random_indices = np.random.randint(0, X_train.shape[0], size=batch_size)
    real_images = X_train[random_indices]
    labels = y_train[random_indices]

    generated_images = generator.predict(np.random.randn(batch_size, latent_dim))
    x = np.concatenate([real_images, generated_images])
    y = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    y += 0.05 * np.random.random(y.shape)  # label smoothing

    discriminator_loss  = discriminator.train_on_batch(x, y)

    predictions = discriminator.predict(x)
    discriminator_accuracy = np.mean((predictions >= 0.5).astype(np.float32) == y)

    # Train generator
    noise = np.random.randn(batch_size, latent_dim)
    misleading_labels = np.ones((batch_size, 1))

    generator_loss = gan.train_on_batch(noise, misleading_labels)

    # Calculate discriminator loss on generated samples
    generated_images = generator.predict(noise)
    discriminator_loss_generated = discriminator.test_on_batch(generated_images, np.zeros((batch_size, 1)))

    # Print progress
    if epoch % 20 == 0:
        print(f"Epoch: {epoch} Discriminator Loss: {discriminator_loss} Generator Loss: {generator_loss}\n",f"Discriminator Accuracy: {discriminator_accuracy} Generated Loss: {discriminator_loss_generated}\n" )

4/4 [==============================] - 1s 350ms/step
Epoch: 0 Discriminator Loss: 0.6973707675933838 Generator Loss: 0.7104899883270264
 Discriminator Accuracy: 0.0 Generated Loss: 0.6833667159080505

4/4 [==============================] - 1s 213ms/step
Epoch: 20 Discriminator Loss: 0.6957773566246033 Generator Loss: 0.7042207717895508
 Discriminator Accuracy: 0.0 Generated Loss: 0.6833667159080505

4/4 [==============================] - 1s 211ms/step
Epoch: 40 Discriminator Loss: 0.7033243775367737 Generator Loss: 0.7105344533920288
 Discriminator Accuracy: 0.0 Generated Loss: 0.6833667159080505

4/4 [==============================] - 1s 217ms/step
Epoch: 60 Discriminator Loss: 0.6969490051269531 Generator Loss: 0.7164795994758606
 Discriminator Accuracy: 0.0 Generated Loss: 0.6833667159080505

4/4 [==============================] - 1s 215ms/step
Epoch: 80 Discriminator Loss: 0.7035359740257263 Generator Loss: 0.7086350321769714
 Discriminator Accuracy: 0.0 Generated Loss: 0.683366715

In [ ]:
X_test = X_test.reshape(-1, 28, 28, 1)
evaluation_result= discriminator.evaluate(X_test, y_test)
print(evaluation_result)

294/294 [==============================] - 21s 70ms/step - loss: 0.7755
0.775546133518219
